## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-30-20-53-00 +0000,wsj,Opinion | Cornell University Discriminated Aga...,https://www.wsj.com/opinion/cornell-university...
1,2025-07-30-20-52-36 +0000,nyt,U.S. Hits Brazil With 50% Tariffs and Sanction...,https://www.nytimes.com/2025/07/30/world/ameri...
2,2025-07-30-20-52-01 +0000,nypost,Trump targets New York drug ‘injection sites’ ...,https://nypost.com/2025/07/30/us-news/trump-ta...
3,2025-07-30-20-52-00 +0000,wsj,Opinion | Generation Z Walks Into a Bar—and Gr...,https://www.wsj.com/opinion/generation-z-walks...
4,2025-07-30-20-52-00 +0000,nyt,N.T.S.B. Details Evidence in Fatal Crash at Re...,https://www.nytimes.com/2025/07/30/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,55
190,tsunami,26
13,new,19
111,california,17
462,gaza,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
262,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...,112
163,2025-07-30-14-30-00 +0000,wsj,The Senate confirmed former Trump defense atto...,https://www.wsj.com/politics/policy/trumps-for...,111
124,2025-07-30-16-33-22 +0000,bbc,Trump to hit India with 25% tariffs - plus 'pe...,https://www.bbc.com/news/articles/cz71w1dgl84o...,107
313,2025-07-29-22-00-59 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...,104
319,2025-07-29-21-42-41 +0000,nypost,Top Zelensky official praises Trump for giving...,https://nypost.com/2025/07/29/us-news/top-zele...,104


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
262,112,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...
33,88,2025-07-30-20-19-00 +0000,wsj,Former Vice President Kamala Harris said Wedne...,https://www.wsj.com/politics/elections/kamala-...
240,75,2025-07-30-04-38-58 +0000,bbc,Watch: Major earthquake off Russia triggers wi...,https://www.bbc.com/news/videos/cwy01l5kgxno?a...
287,58,2025-07-29-23-29-20 +0000,nypost,Zohran Mamdani’s NYC election rivals have litt...,https://nypost.com/2025/07/29/us-news/zohran-m...
139,55,2025-07-30-16-06-00 +0000,wsj,Any further Bank of Canada rate reduction depe...,https://www.wsj.com/articles/bank-of-canada-st...
70,54,2025-07-30-19-20-30 +0000,nypost,Israel suggests it could annex parts of Gaza i...,https://nypost.com/2025/07/30/world-news/israe...
270,46,2025-07-30-01-02-00 +0000,wsj,Possible dissents this week by Christopher Wal...,https://www.wsj.com/economy/central-banking/fe...
103,43,2025-07-30-18-13-51 +0000,nypost,Fed keeps rates unchanged — but two vote again...,https://nypost.com/2025/07/30/business/fed-kee...
146,38,2025-07-30-15-55-13 +0000,nyt,"NISAR, a NASA-India Collaboration, Will Study ...",https://www.nytimes.com/2025/07/29/science/nis...
83,36,2025-07-30-19-00-58 +0000,nypost,Legal NY pot sellers fume over latest Albany c...,https://nypost.com/2025/07/30/us-news/irate-ca...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
